In [1]:
#!pip install transformers

In [2]:
#!pip install sentence-transformers

In [5]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

2021-11-27 12:56:00.983286: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64:/usr/local/apps/cuDNN/7.6.5-cuda-10.2/lib64
2021-11-27 12:56:00.983325: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [7]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [8]:
import pandas as pd
citants = pd.read_json('From-ScisummNet-2019/A00-1031/citing_sentences.json')

In [9]:
citants = citants[['citance_No','clean_text']]
citants
queries = list(citants['clean_text'])
# cit_no = list(citants['citance_No'])
# query_sent = citants['raw_text']
# query_sent

In [10]:
cite_no = list(citants.citance_No)

In [11]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
tree = ET.parse("From-ScisummNet-2019/A00-1031"+"/Reference_XML/"+"A00-1031"+".xml")
root = tree.getroot()
final1=[]
final2=[]
i = 0
total = len(root)
# print(total)
for a in root:
  #  print(a.attrib)
   # print(i)
    for b in a:

        final1.append(b.text)
        if i == 0:
            final2.append("Abstract")
        if i == 1:
            final2.append("Introduction")
        elif i < total-2:
            final2.append("Experiment/Discussion")
        if i == total-2 or i == total-1:
            final2.append("Results/Conclusion")
        if i == total:
            final2.append("Acknowledgment")
    i = i+1


d={'col1':final1,'col2':final2}
rp = pd.DataFrame(data=d)

In [12]:
rp.head

<bound method NDFrame.head of                                                   col1                col2
0    Trigrams'n'Tags (TnT) is an efficient statisti...        Introduction
1    Contrary to claims found elsewhere in the lite...        Introduction
2    A recent comparison has even shown that TnT pe...        Introduction
3    We describe the basic model of TnT, the techni...        Introduction
4    Furthermore, we present evaluations on two cor...        Introduction
..                                                 ...                 ...
173  Many thanks go to Hans Uszkoreit for his suppo...  Results/Conclusion
174  Most of the work on TnT was carried out while ...  Results/Conclusion
175  Large annotated corpora are the pre-requisite ...  Results/Conclusion
176  Therefore, I would like to thank all the peopl...  Results/Conclusion
177  And, last but not least, I would like to thank...  Results/Conclusion

[178 rows x 2 columns]>

In [13]:
rp.col1

0      Trigrams'n'Tags (TnT) is an efficient statisti...
1      Contrary to claims found elsewhere in the lite...
2      A recent comparison has even shown that TnT pe...
3      We describe the basic model of TnT, the techni...
4      Furthermore, we present evaluations on two cor...
                             ...                        
173    Many thanks go to Hans Uszkoreit for his suppo...
174    Most of the work on TnT was carried out while ...
175    Large annotated corpora are the pre-requisite ...
176    Therefore, I would like to thank all the peopl...
177    And, last but not least, I would like to thank...
Name: col1, Length: 178, dtype: object

In [14]:
ann = "From-ScisummNet-2019/A00-1031/annotation/A00-1031.ann.txt"
with open(ann,"r") as f:
    data = f.read()

In [15]:
idx = {'Citance Number':0,'Reference Article':1,'Citing Article':2,'Citation Marker Offset':3,'Citation Marker':4,'Citation Offset':5,'Reference Offset':7}

In [16]:
gt = {}
lines = data.split('\n')
for line in lines:
    lis = line.split('|')
#     print(lis, len(lis))
    if len(lis)==11:
            cit_no = lis[0].split(':')[1]
            ref_off = lis[7].split(':')[1].strip()
            exec("ref_off="+ref_off)
            ref_off = list(map(int, ref_off))
            gt[int(cit_no)] = ref_off #

In [17]:
gt

{1: [0, 36],
 2: [44, 178],
 3: [15, 173],
 4: [0, 1],
 5: [27, 165],
 6: [27, 141],
 7: [115, 154],
 8: [24, 165],
 9: [36, 46],
 10: [2, 75],
 11: [0, 1],
 12: [18, 40],
 13: [36, 118],
 14: [24, 173],
 15: [79, 80],
 16: [0, 113],
 17: [12, 19],
 18: [0, 1],
 19: [28, 157],
 20: [36, 164]}

In [18]:
# def clean(line):
#     line = line.split("|")
#     cit_2 , ref_2 = line[6].split("Citation Text:")[1].strip() , line[8].split("Reference Text:")[1].strip()
#     cit_2 = cit_2.split(" >")[1].split("</S>")[0]
#     ref_2 = ref_2.split(" >")[1:]
#     ref_2 = [ i.split("</S>")[0] for i in ref_2]
#     ans=""
#     for i in ref_2:
#         ans=ans + cit_2 + " $$$$$ " + i +"\n"
#    # ref_2 = " ".join(ref_2)
    
#     return ans

In [19]:
# with open('./From-ScisummNet-2019/A00-1031/annotation/A00-1031.ann.txt') as f:
#     data = f.read()
# data = [ clean(i) for i in data.split("\n") if i ]

# # with open("cleaned.txt","w+") as f:
# #     f.write("\n".join(data))

In [20]:
corpus = rp.col1

In [21]:
cite_no

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [22]:
import scipy
import numpy as np

In [23]:
def get_matching_sentences(model):
    # Get a vector for each headline (sentence) in the corpus
    corpus_embeddings = model.encode(corpus)
    # Define search queries and embed them to vectors as well

    query_embeddings = model.encode(queries)
    # For each search term return 5 closest sentences
    closest_n = 5

    for i in range(len(queries)):
        query, query_embedding  = queries[i], query_embeddings[i]
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
    #     results = dict(enumerate(distances,1))
    #     results = dict(sorted(results.items(), key=lambda item: item[1]))

        print("\n\n======================\n\n")
        print("Query:", query)
        print("\nTop 5 most similar sentences in corpus:")
        indexes = results[0:closest_n]
        top_n = []
        for l,k in indexes:
            top_n.append(l)
        print(np.intersect1d(top_n,gt[cite_no[i]]))
        print("indexes{}, top_n{}, i{}, cite_no[i]{} ,gt[cite_no[i]]{}".format(indexes, top_n,i,cite_no[i],gt[cite_no[i]]))

    #     for idx, distance in results[0:closest_n]:
    #         print(corpus[idx].strip(), "(Score: %.4f)" % (1-distance))


In [24]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

# #Define your train examples. You need more than just two examples...

# train_examples = [InputExample(texts=['My first sentence', 'My second sentence'], label=0.8),
#     InputExample(texts=['Another pair', 'Unrelated sentence'], label=0.3)]

# #Define your train dataset, the dataloader and the train loss
# train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# train_loss = losses.CosineSimilarityLoss(model)

# #Tune the model
# model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10, warmup_steps=100)

In [25]:
# data = []
# for i in range(len(queries)):
#     for j in gt[cite_no[i]]:
#         if j < len(corpus):
# #             print(queries[i],gt[cite_no[i]],corpus[j])
#             data.append(InputExample(texts=[queries[i],corpus[j]],label=0.8))
    
        

In [26]:
# #Define your train dataset, the dataloader and the train loss
# train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# train_loss = losses.CosineSimilarityLoss(model)

# #Tune the model
# model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10000, warmup_steps=100)

In [23]:
get_matching_sentences(model)





Query: The sentences in the DSO collection were tagged with parts of speech using TnT (Brants, 2000) trained on the Brown Corpus itself.

Top 5 most similar sentences in corpus:
[]
indexes[(16, 0.27199719780161213), (3, 0.2751179495764202), (26, 0.3061739845249595), (33, 0.32632695556253266), (174, 0.33747301984757716)], top_n[16, 3, 26, 33, 174], i0, cite_no[i]1 ,gt[cite_no[i]][0, 36]




Query: 

Top 5 most similar sentences in corpus:
[]
indexes[(34, 0.20756660002426197), (70, 0.28087194268629), (65, 0.3711071328983985), (57, 0.38959269651066464), (61, 0.4191898053080748)], top_n[34, 70, 65, 57, 61], i1, cite_no[i]2 ,gt[cite_no[i]][44, 178]




Query: The English POS-tagging has been carried out using freely available TNT tagger (Brants, 2000).

Top 5 most similar sentences in corpus:
[]
indexes[(3, 0.31457500053583565), (16, 0.3492840588830657), (172, 0.37751118188939703), (0, 0.381270112530814), (26, 0.4047158733168057)], top_n[3, 16, 172, 0, 26], i2, cite_no[i]3 ,gt[cite_no[i

In [24]:
import xgboost as xgb

In [25]:
from sklearn.ensemble import GradientBoostingClassifier

In [39]:
# clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(X_train, y_train)

In [30]:
!pip install pytorch-ignite

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 240 kB 2.1 MB/s eta 0:00:01


In [38]:
# def get_matching_sentences(model):
# Get a vector for each headline (sentence) in the corpus
# from ignite.metrics import Rouge
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

# m = Rouge(variants=["L", 2], multiref="best")
corpus_embeddings = model.encode(corpus)
# Define search queries and embed them to vectors as well

query_embeddings = model.encode(queries)
# For each search term return 5 closest sentences
closest_n = 5

# for i in range(len(queries)):
i = 0
query, query_embedding  = queries[i], query_embeddings[i]
distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

results = zip(range(len(distances)), distances)
results = sorted(results, key=lambda x: x[1])
#     results = dict(enumerate(distances,1))
#     results = dict(sorted(results.items(), key=lambda item: item[1]))

print("\n\n======================\n\n")
print("Query:", query)
print("\nTop 5 most similar sentences in corpus:")
indexes = results[0:closest_n]
top_n = []
for l,k in indexes:
    top_n.append(l)
print(np.intersect1d(top_n,gt[cite_no[i]]))

print("indexes{}, top_n{}, i{}, cite_no[i]{} ,gt[cite_no[i]]{}".format(indexes, top_n,i,cite_no[i],gt[cite_no[i]]))

for idx, distance in results[0:closest_n]:
#     m.update(([query.split()],[corpus[idx].strip().split()]))
#     print(m.compute())
    scores = scorer.score(corpus[idx].strip(), query)
    print(scores)
    print(corpus[idx].strip(), "(Score: %.4f)" % (1-distance))







Query: The sentences in the DSO collection were tagged with parts of speech using TnT (Brants, 2000) trained on the Brown Corpus itself.

Top 5 most similar sentences in corpus:
[]
indexes[(16, 0.27199719780161213), (3, 0.2751179495764202), (26, 0.3061739845249595), (33, 0.32632695556253266), (174, 0.33747301984757716)], top_n[16, 3, 26, 33, 174], i0, cite_no[i]1 ,gt[cite_no[i]][0, 36]
{'rouge1': Score(precision=0.22727272727272727, recall=0.35714285714285715, fmeasure=0.2777777777777778), 'rougeL': Score(precision=0.18181818181818182, recall=0.2857142857142857, fmeasure=0.2222222222222222)}
This paper describes the models and techniques used by TnT together with the implementation. (Score: 0.7280)
{'rouge1': Score(precision=0.22727272727272727, recall=0.29411764705882354, fmeasure=0.25641025641025644), 'rougeL': Score(precision=0.18181818181818182, recall=0.23529411764705882, fmeasure=0.20512820512820512)}
We describe the basic model of TnT, the techniques used for smoothing and f

In [ ]:
for k in os.listdir("./From-ScisummNet-2019"):
    try:

In [32]:
m = Rouge(variants=["L", 2], multiref="best")

candidate = "the cat is not there".split()
references = [
    "the dogs are on the mat".split()
]

m.update(([candidate], [references]))

m.compute()

{'Rouge-L-P': 0.2,
 'Rouge-L-R': 0.16666666666666666,
 'Rouge-L-F': 0.16666666666666666,
 'Rouge-2-P': 0.0,
 'Rouge-2-R': 0.0,
 'Rouge-2-F': 0.0}

In [33]:
!pip install rouge-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:

scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')

In [35]:
scores

{'rouge1': Score(precision=0.75, recall=0.6666666666666666, fmeasure=0.7058823529411765),
 'rougeL': Score(precision=0.625, recall=0.5555555555555556, fmeasure=0.5882352941176471)}